### Alter and update EMPIRE Input - Node.xlsx

*Some differences between OpenEMPIRE europe_v51 (most recent; base) and EMPIRE-Public ("Latitude" and "Longitude" cols)*

In [2]:
import pandas as pd
import numpy as np

#### Tab: ElectricAnnualDemand 

Keep as is

#### Tab: NodeLostLoadCost

Keep as is

#### Tab: HydroGenMaxAnnualProduction

Add newly created nodes (max prod = 0)

In [4]:
hydrogen_max_annual_prod_tab = pd.read_excel("Data_OpenEMPIRE/Node.xlsx", sheet_name="HydroGenMaxAnnualProduction", header=2)
hydrogen_max_annual_prod_tab

,Nodes,HydroGenMaxAnnualProduction in MWh per year
0,Austria,45475000.0
1,Bosnia H,5694540.0
2,Belgium,470800.0
3,Bulgaria,3971840.0
4,Switzerland,40660000.0
5,Czech R,2433180.0
6,Germany,23540000.0
7,Denmark,33170.0
8,Estonia,32100.0
9,Spain,44940000.0


In [11]:
ow_nodes = pd.read_csv("Output/OW_nodes.csv")[["Node"]].rename(columns={"Node": "Nodes"})
ow_nodes["HydroGenMaxAnnualProduction in MWh per year"] = 0
hydrogen_max_annual_prod_tab = pd.concat([hydrogen_max_annual_prod_tab, ow_nodes], ignore_index=True)
hydrogen_max_annual_prod_tab

,Nodes,HydroGenMaxAnnualProduction in MWh per year
0,Austria,45475000.0
1,Bosnia H,5694540.0
2,Belgium,470800.0
3,Bulgaria,3971840.0
4,Switzerland,40660000.0
...,...,...
62,Nordavind A,0.0
63,Nordavind D,0.0
64,Nordavind C,0.0
65,Vestavind E,0.0


#### Tab: Latitude

In [13]:
nodes = pd.read_csv("Output/nodes.csv")
latitude_tab = nodes[["Node", "Latitude"]]
latitude_tab

,Node,Latitude
0,Austria,47.460000
1,Bosnia H,44.060000
2,Belgium,50.720000
3,Bulgaria,42.710000
4,Switzerland,46.820000
...,...,...
62,Nordavind A,71.113248
63,Nordavind D,71.450117
64,Nordavind C,71.728123
65,Vestavind E,59.109692


#### Tab: Longitude

In [14]:
longitude_tab = nodes[["Node", "Longitude"]]
longitude_tab

,Node,Longitude
0,Austria,14.430000
1,Bosnia H,17.810000
2,Belgium,4.600000
3,Bulgaria,25.250000
4,Switzerland,8.150000
...,...,...
62,Nordavind A,32.012239
63,Nordavind D,18.808962
64,Nordavind C,20.019911
65,Vestavind E,3.853030


### Write to XLS

In [15]:
SHEET_TO_DF_AND_ROW = dict({
    "HydroGenMaxAnnualProduction": [hydrogen_max_annual_prod_tab, 4],
    "Latitude": [latitude_tab, 4],
    "Longitude": [longitude_tab, 4],
})

In [16]:
# Clean and overwrite existing input_file from original
import shutil

source = "Data_OpenEMPIRE/Node.xlsx"
destination = "EMPIRE_input/Node.xlsx"

shutil.copy(source, destination)

'EMPIRE_input/Node.xlsx'

In [25]:
import openpyxl as ox

def update_spreadsheet(path:str, sheet_name:str, _df : pd.DataFrame, startcol:int=1, startrow:int=4):
    wb = ox.load_workbook(path)
    try: 
        ws=wb[sheet_name]
    except KeyError:
        ws = wb.create_sheet(sheet_name)
    
    # Write headers
    idx = 0
    for col in _df.columns:
        ws.cell(row = startrow - 1, column = startcol + idx).value = col
        idx += 1

    for row in range(0, _df.shape[0]): #For each row in the dataframe
        for col in range(0, _df.shape[1]): #For each column in the dataframe
            ws.cell(row = startrow + row, column = startcol + col).value = _df.iat[row, col]
    wb.save(path)

In [26]:
#Call the defined function
for sheet, df_and_row in SHEET_TO_DF_AND_ROW.items():
    df = df_and_row[0]
    row = df_and_row[1]
    update_spreadsheet(destination, sheet, df, startrow=row)